<a href="https://colab.research.google.com/github/giruffo/Cantate/blob/main/cantate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from datetime import date, timedelta

# --- 1. MOTORE LOGICO CORRETTO ---
def calculate_easter(year: int) -> date:
    a, b, c = year % 19, year // 100, year % 100
    d = (19 * a + b - b // 4 - ((b - (b + 8) // 25 + 1) // 3) + 15) % 30
    e = (32 + 2 * (b % 4) + 2 * (c // 4) - d - (c % 4)) % 7
    f = d + e - 7 * ((a + 11 * d + 22 * e) // 451) + 114
    return date(year, f // 31, f % 31 + 1)

def get_liturgical_dates(year: int) -> dict[date, str]:
    easter = calculate_easter(year)
    xtmas = date(year, 12, 25)
    adv4 = xtmas - timedelta(days=(xtmas.weekday() + 1) % 7 or 7)
    adv1 = adv4 - timedelta(weeks=3)

    dates = {
        date(year, 1, 1): "New Year's Day", date(year, 1, 6): "Epiphany",
        date(year, 12, 25): "Christmas Day", date(year, 12, 26): "Second Day of Christmas",
        date(year, 12, 27): "Third Day of Christmas", easter: "Easter Sunday",
        easter - timedelta(days=7): "Palm Sunday", easter - timedelta(days=2): "Good Friday",
        date(year, 10, 31): "Reformation", date(year, 9, 29): "St. Michael's Day"
    }

    # Domenica dopo Capodanno (Importante per il 4 Gennaio 2026)
    sun_ny = date(year, 1, 1) + timedelta(days=(6-date(year,1,1).weekday()) or 7)
    if sun_ny < date(year, 1, 6): dates[sun_ny] = "Sunday after New Year"

    for i, n in enumerate(['I','II','III','IV']):
        dates[adv1 + timedelta(weeks=i)] = f"Advent {n}"

    sep = easter - timedelta(days=63)
    dates[sep], dates[sep + timedelta(days=7)], dates[sep + timedelta(days=14)] = "Septuagesima", "Sexagesima", "Estomihi"

    curr, n = date(year, 1, 6) + timedelta(days=(6 - date(year, 1, 6).weekday()) or 7), 1
    while curr < sep:
        dates[curr] = f"Epiphany {n}"
        curr += timedelta(days=7)
        n += 1

    pnames = ["Quasimodogeniti", "Misericordias Domini", "Jubilate", "Cantate", "Rogate", "Exaudi"]
    for i, name in enumerate(pnames): dates[easter + timedelta(weeks=i+1)] = name

    curr, n = (easter + timedelta(days=56)) + timedelta(days=7), 1
    while curr < adv1:
        dates[curr] = f"Trinity {n}"
        curr += timedelta(days=7)
        n += 1
    return dates

# --- 2. DATABASE INTEGRALE ---
db = {
    'Christmas Day': [('248/I','Jauchzet, frohlocket!')],
    'Second Day of Christmas': [('40','Dazu ist erschienen'), ('121','Christum wir sollen')],
    'New Year\'s Day': [('190','Singet dem Herrn'), ('41','Jesu, nun sei gepreiset'), ('248/IV','Fallt mit Danken')],
    'Sunday after New Year': [('153','Schau, lieber Gott'), ('58','Ach Gott, wie manches'), ('248/V','Ehre sei dir, Gott')],
    'Epiphany': [('65','Sie werden aus Saba'), ('123','Liebster Immanuel'), ('248/VI','Herr, wenn die stolzen')],
    'Trinity 27': [('140','Wachet auf, ruft uns la voce')]
    # (Il database può essere espanso con tutte le altre 200 cantate qui)
}

# --- 3. MOTORE DI ANALISI ---
def analyze_week(target_date: date):
    lit_dates = get_liturgical_dates(target_date.year)
    monday = target_date - timedelta(days=target_date.weekday())
    print(f"\n🎹 BACH COMPANION - Settimana del {target_date.year}")
    for i in range(7):
        current = monday + timedelta(days=i)
        occ = lit_dates.get(current, "Feria")
        print(f"🗓️ {current.strftime('%a %d %b')} -> {occ}")

        if (current.month == 12 and current.day >= 25) or (current.month == 1 and current.day <= 6):
            print("   💡 [Periodo Oratorio di Natale BWV 248]")

        for bwv, title in db.get(occ, []):
            print(f"   🎼 BWV {bwv}: {title}")
        print("-" * 40)

# ESECUZIONE
analyze_week(date.today())


🎹 BACH COMPANION - Settimana del 2026
🗓️ Mon 29 Dec -> Feria
   💡 [Periodo Oratorio di Natale BWV 248]
----------------------------------------
🗓️ Tue 30 Dec -> Feria
   💡 [Periodo Oratorio di Natale BWV 248]
----------------------------------------
🗓️ Wed 31 Dec -> Feria
   💡 [Periodo Oratorio di Natale BWV 248]
----------------------------------------
🗓️ Thu 01 Jan -> New Year's Day
   💡 [Periodo Oratorio di Natale BWV 248]
   🎼 BWV 190: Singet dem Herrn
   🎼 BWV 41: Jesu, nun sei gepreiset
   🎼 BWV 248/IV: Fallt mit Danken
----------------------------------------
🗓️ Fri 02 Jan -> Feria
   💡 [Periodo Oratorio di Natale BWV 248]
----------------------------------------
🗓️ Sat 03 Jan -> Feria
   💡 [Periodo Oratorio di Natale BWV 248]
----------------------------------------
🗓️ Sun 04 Jan -> Sunday after New Year
   💡 [Periodo Oratorio di Natale BWV 248]
   🎼 BWV 153: Schau, lieber Gott
   🎼 BWV 58: Ach Gott, wie manches
   🎼 BWV 248/V: Ehre sei dir, Gott
------------------------------